In [1]:
%matplotlib inline
# !!!! IMPORTANT INPUT DATA !!!!!
# NEED TO RE-DOWNLOAD PRECIPITATION, TEMPERATURE, AND ELEVATION FOR ERA INTERIM WITH FINEST GRID SIZE

In [2]:
###############################################################################
"""
Python Glacier Evolution Model "PyGEM" V1.0
Prepared by David Rounce with support from Regine Hock.
This work was funded under the NASA HiMAT project (INSERT PROJECT #).

PyGEM is an open source glacier evolution model written in python.  Model
details come from Radic et al. (2013), Bliss et al. (2014), and Huss and Hock
(2015).
"""
###############################################################################
# This is the main script that provides the architecture and framework for all
# of the model runs. All input data is included in a separate module called
# pygem_input.py. It is recommended to not make any changes to this file unless
# you are a PyGEM developer and making changes to the model architecture.
#
#========== IMPORT PACKAGES ==================================================
# Various packages are used to provide the proper architecture and framework
# for the calculations used in this script. Some packages (e.g., datetime) are
# included in order to speed of calculations and simplify code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os # os is used with re to find name matches
import re # see os
import xarray as xr
import netCDF4 as nc
from time import strftime
import timeit
#========== IMPORT INPUT AND FUNCTIONS FROM MODULES ==========================
import pygem_input as input
import pygemfxns_modelsetup as modelsetup
import pygemfxns_climate as climate
import pygemfxns_massbalance as massbalance
import pygemfxns_output as output

#========== DEVELOPER'S TO-DO LIST ===========================================
# > Output log file, i.e., file that states input parameters, date of model run, model options selected, 
#   and any errors that may have come up (e.g., precipitation corrected because negative value, etc.)

In [3]:
# ===== STEP ONE: Select glaciers included in model run ======================
timestart_step1 = timeit.default_timer()
if input.option_glacier_selection == 1:
    # RGI glacier attributes
    main_glac_rgi = modelsetup.selectglaciersrgitable()
elif input.option_glacier_selection == 2:
    print('\n\tMODEL ERROR (selectglaciersrgi): this option to use shapefiles to select glaciers has not been coded '
          '\n\tyet. Please choose an option that exists. Exiting model run.\n')
    exit()
else:
    # Should add options to make regions consistent with Brun et al. (2017), which used ASTER DEMs to get mass 
    # balance of 92% of the HMA glaciers.
    print('\n\tModel Error (selectglaciersrgi): please choose an option that exists for selecting glaciers.'
          '\n\tExiting model run.\n')
    exit()
timeelapsed_step1 = timeit.default_timer() - timestart_step1
print('Step 1 time:', timeelapsed_step1)

#===== STEP TWO: HYPSOMETRY, ICE THICKNESS, MODEL TIME FRAME, SURFACE TYPE ===
timestart_step2 = timeit.default_timer()

# Glacier hypsometry [km**2]
main_glac_hyps = modelsetup.import_hypsometry(main_glac_rgi)
# Ice thickness [m]
main_glac_icethickness = modelsetup.import_icethickness(main_glac_rgi)
# Add volume [km**3] and mean elevation [m a.s.l.] to the main glaciers table
main_glac_rgi['Volume'], main_glac_rgi['Zmean'] = modelsetup.hypsometrystats(main_glac_hyps, main_glac_icethickness)
# Model time frame
dates_table, start_date, end_date, monthly_columns, annual_columns, annual_divisor = modelsetup.datesmodelrun()
# # Add column with dates as a string depending on timestep (YYYY-MM or YYYY-MM-DD) 
# #  > this is useful for extracting the proper index for the start and end date)
# if timestep == 'monthly':
#     dates_table['date_string'] = dates_table['date'].apply(lambda x: x.strftime('%Y-%m'))
# elif timestep == 'daily':
#     dates_table['date_string'] = dates_table['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
# print(dates_table.head(15))
# Initial surface type
main_glac_surftypeinit = modelsetup.surfacetypeglacinitial(main_glac_rgi, main_glac_hyps)

# Print time elapsed
timeelapsed_step2 = timeit.default_timer() - timestart_step2
print('Step 2 time:', timeelapsed_step2)

print(main_glac_rgi)


This study is only focusing on glaciers ['03473', '03733'] in region [15].
The 'select_rgi_glaciers' function has finished.
Step 1 time: 0.07634434115786888
The 'datesmodelrun' function has finished.
The 'initialsurfacetype' function has finished.
Step 2 time: 4.640578115518114
        O1Index           RGIId     CenLon     CenLat  O1Region  O2Region  \
GlacNo                                                                      
0          3472  RGI60-15.03473  86.715912  28.089468        15         2   
1          3732  RGI60-15.03733  86.903244  27.974716        15         2   

          Area  Zmin  Zmax  Zmed  Slope  Aspect   Lmax    Volume  Zmean  
GlacNo                                                                   
0       61.054  4702  8181  5815   16.5     180  18285  7.057732   5832  
1       19.097  4926  7870  5568   17.5     262  15396  1.976989   5924  


In [4]:
# # Plot glacier hypsometry using matplotlib
# plt.figure(1)
# plt.plot(main_glac_hyps.iloc[0,400:850], main_glac_hyps.columns.values[400:850])
# plt.plot(main_glac_hyps.iloc[1,400:850], main_glac_hyps.columns.values[400:850])
# plt.legend(('Ngozumpa', 'Khumbu'))
# plt.xlabel('Area (km2)')
# plt.ylabel('Elevation (m.a.s.l.)')
# plt.title('Glacier hypsometry (10m bins)')

# # Plot glacier ice thickness
# plt.figure(1)
# plt.plot(main_glac_icethickness.iloc[0,400:850], main_glac_hyps.columns.values[400:850])
# plt.plot(main_glac_icethickness.iloc[1,400:850], main_glac_hyps.columns.values[400:850])
# plt.legend(('Ngozumpa', 'Khumbu'))
# plt.xlabel('Thickness (m)')
# plt.ylabel('Elevation (m.a.s.l.)')
# plt.title('Glacier Thickness (10m bins)')

In [5]:
# Mass balance estimates from High-res DEMs
mb_filepath = '../DEMs/hma_mb_20170717_1846.csv'
# mb_filepath = os.path.dirname(__file__) + '/../DEMs/hma_mb_20170717_1846.csv'
main_glac_mb_raw = pd.read_csv(mb_filepath)
# print(main_glac_mb_raw.head())
print(main_glac_mb_raw.loc[main_glac_mb_raw['# glacnum'] == 15.03473])
# print(main_glac_mb_raw.iloc[8690:8694,:])
# Having trouble finding Khumbu Glacier (mb = -0.7 m w.e. a-1)
# print(main_glac_mb_raw.loc[main_glac_mb_raw['# glacnum'] == 15.03733])
# print(main_glac_mb_raw.shape)
print('\nNeed to extract Khumbu Glacier and automate this process')

      # glacnum          x          y    z_med    z_p16    z_p84  mb_mwea  \
6256   15.03473  166571.02 -892626.41  5695.44  4978.04  6478.77    -0.39   

      area_km2       t1      t2     dt  
6256     61.05  2000.11  2015.0  14.89  

Need to extract Khumbu Glacier and automate this process


In [6]:
#===== STEP THREE: IMPORT CLIMATE DATA ================================================================================
timestart_step3 = timeit.default_timer()
if input.option_gcm_downscale == 1:
    # Air Temperature [degC] and GCM dates
    gcm_glac_temp, gcm_time_series = climate.importGCMvarnearestneighbor_xarray(
            input.gcm_temp_filename, input.gcm_temp_varname, main_glac_rgi, dates_table, start_date, end_date)
    # Air Temperature [degC] and GCM dates
    gcm_glac_prec, gcm_time_series = climate.importGCMvarnearestneighbor_xarray(
            input.gcm_prec_filename, input.gcm_prec_varname, main_glac_rgi, dates_table, start_date, end_date)
    # Elevation [m a.s.l] associated with air temperature data
    gcm_glac_elev = climate.importGCMfxnearestneighbor_xarray(
            input.gcm_elev_filename, input.gcm_elev_varname, main_glac_rgi)
else:
    print('\n\tModel Error: please choose an option that exists for downscaling climate data. Exiting model run now.\n')
    exit()
# Add column of the dates from the gcm data
dates_table['date_gcm'] = gcm_time_series
# Print time elapsed
timeelapsed_step3 = timeit.default_timer() - timestart_step3
print('Step 3 time:', timeelapsed_step3)

The 'importGCMvarnearestneighbor' fxn for 't2m' has finished.
The 'importGCMvarnearestneighbor' fxn for 'tp' has finished.
The 'importGCMfxnearestneighbor' fxn for 'z' has finished.
Step 3 time: 0.08379476325163004


In [7]:
# print('\nSample of glacier temp:\n', gcm_glac_temp.iloc[:,0:5])
# print('\nSample of glacier prec:\n', gcm_glac_prec.iloc[:,0:5])
# print('\nSample of glacier elev:\n', gcm_glac_elev)
# # Plot nearest neighbor meteorological data using matplotlib
# plt.figure(1)
# plt.plot(gcm_glac_temp.columns.values, gcm_glac_temp.iloc[0,:])
# plt.plot(gcm_glac_temp.columns.values, gcm_glac_temp.iloc[1,:])
# plt.legend(('Ngozumpa (gcm_elev: 4229m)', 'Khumbu (gcm_elev: 4037m)'))
# plt.xlabel('Date')
# plt.ylabel('Temperature (deg C)')
# plt.title('Monthly Temperature from GCM nearest neighbor \n(mon_MPI-ESM-LR_rcp85_r1i1p1)')
# # Plot nearest neighbor meteorological data using matplotlib
# plt.figure(2)
# plt.plot(dates_table.loc[:,'date'],gcm_glac_prec.iloc[0,:])
# plt.plot(dates_table.loc[:,'date'],gcm_glac_prec.iloc[1,:])
# plt.xlabel('Date')
# plt.ylabel('Precipitation (m)')
# plt.legend(('Ngozumpa', 'Khumbu'))
# plt.title('Monthly Precipitation from GCM nearest neighbor \n(mon_MPI-ESM-LR_rcp85_r1i1p1)')

In [8]:
gcm_prec_annual_sum = massbalance.groupbyyearsum(gcm_glac_prec)
annual_columns = np.arange(dates_table.loc[0,'wateryear'], dates_table.loc[dates_table.shape[0]-1,'wateryear'] + 1)
gcm_prec_annual_sum.columns = annual_columns
print(gcm_prec_annual_sum)
# Compared to mean annual total of precipitation from 1994-2013 at Pyramid Station (0.449 m)

            2001      2002      2003      2004      2005      2006      2007  \
GlacNo                                                                         
0       1.628995  1.717613  1.838320  1.875830  1.560102  1.691841  2.168342   
1       1.671935  1.794233  1.927749  1.983236  1.657483  1.729570  2.348217   

            2008      2009      2010      2011      2012      2013      2014  \
GlacNo                                                                         
0       1.537144  1.405843  1.545257  1.842922  1.751667  1.823372  1.557914   
1       1.621900  1.523454  1.652764  1.972454  1.836434  1.898143  1.638588   

            2015  
GlacNo            
0       1.651544  
1       1.731744  


In [9]:
# ----- STEP FOUR: CALCULATE SPECIFIC MASS BALANCE --------------------------
""" ADD MODEL RUN ATTRIBUTES - DDF, ETC. TO NETCDF """

' ADD MODEL RUN ATTRIBUTES - DDF, ETC. TO NETCDF '

In [10]:
# Empty dataframes for monthly glacier-wide output (rows = glaciers, columns = dates)
#  for mass balance components, these are the average component over the glacier
# Monthly glacier-wide accumulation [m w.e.]
main_glac_acc_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly glacier-wide refreeze [m w.e.]
main_glac_refreeze_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly glacier-wide melt [m w.e.]
main_glac_melt_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly glacier-wide frontal ablation [m w.e.]
main_glac_frontal_ablation_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly glacier-wide specific mass balance [m w.e.]
main_glac_massbal_clim_mwe_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly total glacier area [km**2]
main_glac_area_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly Equilibrium Line Altitude (ELA) [m a.s.l.]
main_glac_ELA_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly Accumulation-Area Ratio (AAR) [%]
main_glac_AAR_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly Snow Line Altitude [m a.s.l.]
main_glac_snowline_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)
# Monthly runoff [m**3]
main_glac_runoff_monthly = pd.DataFrame(0, index=main_glac_rgi.index, columns=monthly_columns)

# Empty dataframes for annual glacier-wide output (rows = glaciers, columns = years)
# Annual glacier-wide specific accumulation [m w.e.]
main_glac_acc_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual glacier-wide specific refreeze [m w.e.]
main_glac_refreeze_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual glacier-wide specific melt [m w.e.]
main_glac_melt_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual glacier-wide specific frontal ablation [m w.e.]
main_glac_frontal_ablation_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual glacier-wide specific climatic mass balance [m w.e.]
main_glac_massbal_clim_mwe_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual glacier-wide specific total mass balance [m w.e.]
main_glac_massbal_total_mwe_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual glacier area at start of each year [km**2]
main_glac_area_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual glacier volume at start of each year [km**3]
main_glac_volume_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual Equilibrium Line Altitude (ELA) [m a.s.l.]
main_glac_ELA_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual Accumulation-Area Ratio (AAR) [m a.s.l.]
main_glac_AAR_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual snowline altitude [m a.s.l.]
main_glac_snowline_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)
# Annual runoff [m**3]
main_glac_runoff_annual = pd.DataFrame(0, index=main_glac_rgi.index, columns=annual_columns)

In [15]:
start_time = timeit.default_timer()

# for glac in range(main_glac_rgi.shape[0]):
for glac in [1]:
    # Downscale the gcm temperature [degC] to each bin
    glac_bin_temp = massbalance.downscaletemp2bins(main_glac_rgi, main_glac_hyps, gcm_glac_temp, gcm_glac_elev, glac)
    # Downscale the gcm precipitation [m] to each bin (note: not separated into solid and liquid precipitation yet)
    glac_bin_precsnow = massbalance.downscaleprec2bins(main_glac_rgi, main_glac_hyps, gcm_glac_prec, gcm_glac_elev, 
                                                       glac)
    # Compute accumulation [m w.e.] and precipitation [m] for each bin
    glac_bin_prec, glac_bin_acc = massbalance.accumulationbins(glac_bin_temp, glac_bin_precsnow, glac)
    # Compute potential refreeze [m w.e.] for each bin
    glac_bin_refreeze = massbalance.refreezepotentialbins(glac_bin_temp, dates_table)
    
#     # Create empty "glac_bin_" dataframes
#     #  "glac_bin_" dataframes store data of a single glacier: rows are elevation bins, columns are dates
#     # Refreeze [m w.e.]
#     glac_bin_refreeze = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
#     # Surface melt (snow + glacier melt) [m w.e.]
#     glac_bin_melt = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
#     # Frontal ablation [m w.e.]
#     glac_bin_frontal_ablation = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
#     # Specific climatic mass balance [m w.e.]
#     glac_bin_massbal_clim_mwe = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
#     # Area [km**2]
#     glac_bin_area = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
#     # Snow depth [m w.e.]
#     glac_bin_snowdepth = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
#     # Snow melt (only snow on surface) [m w.e.]
#     glac_bin_melt_snowcomponent = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
#     # Glacier melt (only melt of ice/firn after all snow on surface has melted) [m w.e.]
#     glac_bin_melt_glaccomponent = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=monthly_columns)
    
#     # Create empty "glac_bin_..._annual" dataframes for mass balance calculations
#     #  "glac_bin_..._annual" dataframes store data of a single glacier: rows are elevation bins, columns are years
#     # Annual total accumulation [m w.e.]
#     glac_bin_acc_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     # Annual total refreeze [m w.e.]
#     glac_bin_refreeze_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     # Annual total surface melt [m w.e.]
#     glac_bin_melt_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     # Annual total frontal ablation [m w.e.]
#     glac_bin_frontal_ablation_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     # Annual specific climatic mass balance [m w.e.]
#     glac_bin_massbal_clim_mwe_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     # Annual specific total mass balance [m w.e.]
#     glac_bin_massbal_total_mwe_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     # Annual glacier area [km**2]
#     glac_bin_area_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     glac_bin_area_annual.iloc[:,0] = main_glac_hyps.loc[glac,:]
#     # Annual ice thickness [m]
#     glac_bin_icethickness_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     glac_bin_icethickness_annual.iloc[:,0] = main_glac_icethickness.loc[glac,:]
#     # Annual glacier volume [km**3]
#     glac_bin_volume_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     glac_bin_volume_annual.iloc[:,0] = main_glac_hyps.loc[glac,:] * main_glac_icethickness.loc[glac,:] / 1000
#     #  units: [m] * [km**2] * (1 km / 1000 m) = km**3
#     # Annual surface type [1=ice, 2=snow, 3=firn, 4=debris]
#     glac_bin_surftype_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)
#     glac_bin_surftype_annual.iloc[:,0] = main_glac_surftypeinit.loc[glac,:]
#     # Annual glacier width [m]
#     glac_bin_width_annual = pd.DataFrame(0, index=main_glac_hyps.columns.values, columns=annual_columns)


    ###### ENTERING LOOP THROUGH EACH TIME STEP ##############################
    # Loop through each timestep in order to carry over accumulation, which may alter melt rates and surface type
    for step in range(glac_bin_temp.shape[1]):
#     for step in range(0,26):
#     for step in range(0,12):
#         print('Date:',glac_bin_temp.columns.values[step])
        # Extract data associated with each bin for each time step, which will be used in the calculations that follow.  This
        #  improves readability and enables the use of logical indexing on a column by column basis to speed up calculations.
        #  "series_" used to denote data from a single time step and the index is the bins. Furthermore, must use copies to avoid 
        #  SettingWithCopyWarning.
        series_temp = glac_bin_temp.iloc[:,step].copy()
        series_prec = glac_bin_prec.iloc[:,step].copy()
        series_acc = glac_bin_acc.iloc[:,step].copy()
        series_refreeze = glac_bin_refreeze.iloc[:,step].copy()
        series_surftype = glac_bin_surftype_annual.iloc[:,int(step/annual_divisor)].copy()
        # Record the monthly area (currently updates annual, but kept in here for flexibility in future coding)
        glac_bin_area.iloc[:, step] = glac_bin_area_annual.iloc[:, int(step/annual_divisor)].copy()
        series_area = glac_bin_area.iloc[:, step]
        # If monthly timestep, then get the daysinmonth in order to calculate melt
        if timestep == 'monthly':
            series_daysinmonth = dates_table.loc[step,'daysinmonth']
        # Mask cells according to surface type such that only performing calculations on the glacier
        series_temp[series_surftype == 0] = 0
        series_prec[series_surftype == 0] = 0
        series_acc[series_surftype == 0] = 0
        series_refreeze[series_surftype == 0] = 0
        # Update accumulation and precipitation values within the main frame
        glac_bin_acc.iloc[:, step] = series_acc
        glac_bin_prec.iloc[:, step] = series_prec
        # Mask refreezing over ice such that refreezing <= snow depth
        #  (do not apply to first timestep because series_snowdepth doesn't exist yet)
        if step != 0:
            mask_refreeze = ((series_surftype == 1) & (series_refreeze > (series_snowdepth + series_acc)))
            series_refreeze[mask_refreeze] = series_snowdepth[mask_refreeze] + series_acc[mask_refreeze]
            # Note: this was developed for Woodward et al. (1997) computations of refreeze
        # Calculate the accumulation of snow on the surface
        if step == 0:
            series_snowdepth = series_acc + series_refreeze
        else:
            series_snowdepth = glac_bin_snowdepth.iloc[:,step-1] + series_acc + series_refreeze
        # Calculate the melt of snow [m w.e.] and the melt of glacier [m w.e.] if all the snow on the surface has melted
        # Empty the series for the new time step
        series_melt = pd.Series(0, index=series_temp.index)
        series_melt_snowcomponent = pd.Series(0, index=series_temp.index)
        series_melt_glaccomponent = pd.Series(0, index=series_temp.index)
        series_Emelt_remaining = pd.Series(0, index=series_temp.index)
        # Energy (E) [degC day] available for melt is based on the air temperature and number of days
        series_Emelt_available = series_temp * series_daysinmonth
        series_Emelt_available[series_Emelt_available < 0] = 0
        """ DEVELOPER'S NOTE: Option to incorporate daily stdev to estimate melt should be coded here """
        # Calculate energy required to melt all the snow on the surface
        #    (This would be the maximum amount of energy that could be spent melting snow on the surface.  Any additional energy
        #     would melt the underlying ice/firn/snow)
        series_Emelt_snowdepth_max = series_snowdepth / main_glac_parameters.loc[glac,'DDF_snow']
        # Calculate snow melt based on the energy available
        #    Logical indexing will be used for each potential scenario:
        #      (1) Energy available >= Energy required to melt all the snow
        #      (2) Energy available < Energy required to melt all the snow
        # Scenario 1: Energy available >= energy required to melt all the snow on the surface
        #             Leftover energy will melt underlying ice/firn/snow
        mask1 = (series_Emelt_available >= series_Emelt_snowdepth_max)
        # Snow melt is equal to snow on surface
        series_melt_snowcomponent[mask1] = series_snowdepth[mask1]
        # Snow depth [m w.e.] set to zero because all snow has melted
        series_snowdepth[mask1] = 0
        # Remaining energy available is equal to the initial energy available minus the energy used to melt all the snow
        series_Emelt_remaining[mask1] = series_Emelt_available[mask1] - series_Emelt_snowdepth_max[mask1]
        # Mask used to identify cells with energy remaining that now has the glacier ice exposed
        mask2 = ((series_Emelt_remaining > 0) & (series_surftype == 1))
        # Compute the amount of ice melt
        series_melt_glaccomponent[mask2] = series_Emelt_remaining[mask2] * main_glac_parameters.loc[glac, 'DDF_ice']
        # Mask 3-5: if underlying surface type is not ice, then remaining energy goes towards firn/snow/debris
        if option_surfacetype_firn == 0:
            # Energy leftover melts the underlying snow
            mask3 = ((series_Emelt_remaining > 0) & (series_surftype == 2))
            series_melt_glaccomponent[mask3] = series_Emelt_remaining[mask3] * main_glac_parameters.loc[glac, 'DDF_snow']
        elif option_surfacetype_firn == 1:
            # Energy leftover melts the underlying firn
            mask4 = ((series_Emelt_remaining > 0) & (series_surftype == 3))
            series_melt_glaccomponent[mask4] = series_Emelt_remaining[mask4] * main_glac_parameters.loc[glac, 'DDF_firn']
        if option_surfacetype_debris == 1:
            # Energy leftover melts the underlying debris-covered ice
            mask5 = ((series_Emelt_remaining > 0) & (series_surftype == 4))
            series_melt_glaccomponent[mask5] = series_Emelt_remaining[mask5] * main_glac_parameters.loc[glac, 'DDF_debris']
        # Scenario 2: Energy available < Energy required to melt all the snow
        #             Only snow on surface will potentially melt and some snow will remain on the surface
        mask6 = (series_Emelt_available < series_Emelt_snowdepth_max) & (series_Emelt_available > 0)
        # Compute the amount of snow melt
        series_melt_snowcomponent[mask6] = series_Emelt_available[mask6] * main_glac_parameters.loc[glac,'DDF_snow']
        # Update the remaining snow depth [m w.e.], which will carry over to the next timestep
        series_snowdepth[mask6] = series_snowdepth[mask6] - series_melt_snowcomponent[mask6]
        series_Emelt_remaining[mask6] = 0
        # Update the snow depth [m w.e.] for each time step
        glac_bin_snowdepth.iloc[:, step] = series_snowdepth
        # Record surface (snow + glacier), snow (only snow on surface), and glacier (snow/firn/ice beneath snow) melt [m w.e.]
        glac_bin_melt_snowcomponent.iloc[:, step] = series_melt_snowcomponent
        glac_bin_melt_glaccomponent.iloc[:, step] = series_melt_glaccomponent
        series_melt = series_melt_snowcomponent + series_melt_glaccomponent
        glac_bin_melt.iloc[:, step] = series_melt
        # Compute the frontal ablation [m w.e.]
        series_frontal_ablation = glac_bin_frontal_ablation.iloc[:,step].copy()
        #   !!! Add frontal ablation code in here !!!
        # Compute the specific climatic mass balance [m w.e.] for each time step
        glac_bin_massbal_clim_mwe.iloc[:, step] = series_acc + series_refreeze - series_melt - series_frontal_ablation
        #  climatic mass balance = accumulation + refreeze - melt - frontal_ablation
        
        # COMPUTE GLACIER-WIDE STATISTICS
        #  glacier-wide mass balance components equal the sum of the (component of each bin * area_bin) divided by total area
        # Monthly glacier-wide average accumulation [m w.e.]
        main_glac_acc_monthly.iloc[glac, step] = (glac_bin_acc.iloc[:, step] * series_area).sum() / series_area.sum()
        # Monthly glacier-wide average refreeze [m w.e.]
        main_glac_refreeze_monthly.iloc[glac, step] = (glac_bin_refreeze.iloc[:, step] * series_area).sum() / series_area.sum()
        # Monthly glacier-wide average melt [m w.e.]
        main_glac_melt_monthly.iloc[glac, step] = (glac_bin_melt.iloc[:, step] * series_area).sum() / series_area.sum()
        # Monthly glacier-wide average melt [m w.e.]
        main_glac_frontal_ablation_monthly.iloc[glac, step] = ((glac_bin_melt.iloc[:, step] * series_area).sum() / 
                                                               series_area.sum())
        # Monthly glacier-wide specific climatic mass balance [m w.e.]
        main_glac_massbal_clim_mwe_monthly.iloc[glac, step] = ((glac_bin_massbal_clim_mwe.iloc[:, step] * series_area).sum() / 
                                                               series_area.sum())
        # Monthly glacier area
        main_glac_area_monthly.iloc[glac, step] = series_area.sum()
        # Monthly Equilibrium Line Altitude (ELA) [m a.s.l.]
        series_massbal_clim_mwe = glac_bin_massbal_clim_mwe.iloc[:, step]
        # set up past value in case mass balance is zero over the entire glacier (acc = 0, temp < 0)
        if step == 0:
            ELA_monthly_past = -9.99
        else:
            ELA_monthly_past = main_glac_ELA_monthly.iloc[glac, step-1]
        ELA_monthly_value = massbalance.ELA_glacier(series_massbal_clim_mwe, ELA_monthly_past)
        main_glac_ELA_monthly.iloc[glac, step] = ELA_monthly_value
        # Monthly Accumulation-Area Ratio (AAR) [%]
        #  Note: this function requires the ELA to be known
        main_glac_AAR_monthly.iloc[glac, step] = massbalance.AAR_glacier(ELA_monthly_value, series_area, glac)
        # Transient Snow Line [m a.s.l.]
        #   > the line separating snow from ice or firn at any time, i.e., the lowest elevation/bin that has snow on its surface
        main_glac_snowline_monthly.iloc[glac, step] = series_snowdepth.index.values[np.where((series_snowdepth > 0))[0][0]]
        #  np.where((series_snowdepth > 0)) returns an array of index positions where there is snow on the surface. Since we 
        #   want the minimum/lowest, select [0] to get the values of the array, then [0] again to select the first element
        #  series_snowdepth.index.values[] then returns the desired minimum snow line altitude
        # Monthly Glacier Runoff [m**3]
        main_glac_runoff_monthly.iloc[glac, step] = ((series_melt - series_refreeze + series_prec) * series_area * 1000**2).sum()
            #  runoff = Sum of [(melt - refreeze + precipitation) * Area_bin]
            #  units: m w.e. * km**2 * (1000 m / 1 km)**2 = [m**3]

        ###### ENTERING ANNUAL LOOP ####################################
        # At the end of each year compute annual statistics for bookkeeping and update/calculate glacier characteristics
        if (step + 1) % annual_divisor == 0:
            #  % gives the remainder. Since step starts at 0, add 1 such that at the end of the year, the statistics are 
            #  computed and annual updates performed (this works regardless of being the water or calendar year)
            # Compute the yearly position, which will be used for indexing
            year_position = int(step/annual_divisor)
            # For calculating annual mass balance components below,
            #   year_position*annual_divisor gives initial step of the given year
            #   step gives the final step of the year and the "+1" is needed to include the final step (python is exclusive)
            # Compute the annual accumulation [m w.e.] for each bin
            glac_bin_acc_annual.iloc[:, year_position] = glac_bin_acc.iloc[:, year_position*annual_divisor:step+1].sum(axis=1)
            # Compute the annual refreeze [m w.e.] for each bin
            glac_bin_refreeze_annual.iloc[:, year_position] = (
                glac_bin_refreeze.iloc[:, year_position*annual_divisor:step+1].sum(axis=1))
            # Compute the annual surface melt [m w.e.] for each bin
            glac_bin_melt_annual.iloc[:, year_position] = glac_bin_melt.iloc[:, year_position*annual_divisor:step+1].sum(axis=1)
            # Compute the annual frontal ablation [m w.e.] for each bin
            glac_bin_frontal_ablation_annual.iloc[:, year_position] = (
                glac_bin_frontal_ablation.iloc[:, year_position*annual_divisor:step+1].sum(axis=1))
            # Compute the annual specific climatic mass balance [m w.e.] for each bin
            glac_bin_massbal_clim_mwe_annual.iloc[:, year_position] = (
                glac_bin_massbal_clim_mwe.iloc[:, year_position*annual_divisor:step+1].sum(axis=1))
            #
            ###### Surface Type (turn into separate function) #####
            # Compute the surface type for each bin
            #   Next year's surface type is based on the bin's average annual specific climatic mass balance over the last 5
            #   years. If less than 5 years, then use the average specific climatic mass balance of the existing years
            if year_position < 5:
                # Calculate the average annual specific climatic mass balance [m w.e.] since the model began
                series_surftype_massbal_clim_mwe_runningavg = (
                    glac_bin_massbal_clim_mwe_annual.iloc[:,0:year_position+1].mean(axis=1))
            # Otherwise, use 5 year average annual specific climatic mass balance [m w.e.]
            # if statement used to avoid errors with the last year, since the model run is over
            elif glac_bin_surftype_annual.columns.values[year_position] != endyear:
                series_surftype_massbal_clim_mwe_runningavg = (
                    glac_bin_massbal_clim_mwe_annual.iloc[:, year_position-4:year_position+1].mean(axis=1))
                #  "year_position - 4 : year_position + 1" provides 5 year running average with current year included
            # If the average annual specific climatic mass balance is negative, then the surface type is ice (or debris)
            mask_surftype_accumulationneg = (series_surftype_massbal_clim_mwe_runningavg <= 0) & (series_surftype > 0)
            series_surftype[mask_surftype_accumulationneg] = 1
            # If the average annual specific climatic mass balance is positive, then the surface type is snow (or firn)
            mask_surftype_accumulationpos = (series_surftype_massbal_clim_mwe_runningavg > 0) & (series_surftype > 0)
            series_surftype[mask_surftype_accumulationpos] = 2
            # Apply model options for firn and debris
            if option_surfacetype_firn == 1:
                series_surftype[series_surftype == 2] = 3
                #  if firn is included in the model, then change any snow surface to firn
            if option_surfacetype_debris == 1:
                print('Need to code the model to include debris. This option does not '
                      'currently exist.  Please choose an option that does.\nExiting '
                      'the model run.')
                exit()
            # Update the surface type for the following year
            # if statement used to avoid error in final year, since model run is over
            if glac_bin_surftype_annual.columns.values[year_position] != endyear:
                glac_bin_surftype_annual.iloc[:,year_position+1] = series_surftype
            #    
            ###### GLACIER GEOMETRY CHANGE (turn into separate function) ######
            # Glacier Area / Ice Thickness
            # For calibration run (option_modelruntype = 0), area is constant while ice thickness is changed
            if option_modelrun_type == 0:
                # If statement used to avoid error in final year, since model run is over
                if glac_bin_area_annual.columns.values[year_position] != endyear:
                    # glacier area [km**2] remains constant
                    glac_bin_area_annual.iloc[:, year_position + 1] = glac_bin_area_annual.iloc[:, year_position]
                    # glacier ice thickness [m] changes according to specific climatic mass balance
                    glac_bin_icethickness_annual.iloc[:, year_position + 1] = (
                        glac_bin_icethickness_annual.iloc[:, year_position] + 
                        glac_bin_massbal_clim_mwe_annual.iloc[:, year_position]
                        )
            # For future model runs, area and ice thickness can change
            elif option_modelrun_type == 1:
                # If statement used to avoid error in final year, since model run is over
                if glac_bin_area_annual.columns.values[year_position] != endyear:
                    # glacier area [km**2] remains constant
                    glac_bin_area_annual.iloc[:, year_position + 1] = glac_bin_area_annual.iloc[:, year_position]
                    
                    # NOTE THE AREA NEEDS TO BE UPDATED !!!
                    
                # Option 1 (default) for geometry change, Huss and Hock (2015)
                if option_geometrychange == 1:
                    #Select the factors for the normalized ice thickness change curve based on glacier size
                    if glac_bin_area_annual.iloc[:, year_position].sum() > 20:
                        [gamma, a, b, c] = [6, -0.02, 0.12, 0]
                    elif glac_bin_area_annual.iloc[:, year_position].sum() > 5:
                        [gamma, a, b, c] = [4, -0.05, 0.19, 0.01]
                    else:
                        [gamma, a, b, c] = [2, -0.30, 0.60, 0.09]
                    # Compute the normalized elevation range [-]
                    series_elevrange_norm = ((series_area.index.values[np.where(series_area > 0)[0][-1]] - 
                                                series_area.index.values) / 
                                               (series_area.index.values[np.where(series_area > 0)[0][-1]] - 
                                                series_area.index.values[np.where(series_area > 0)[0][0]]))
                    #  (max_elev - bin_elevation) / (max_elev - min_elev)
                    # Compute the normalized icethickness change [-]
                    series_icethicknesschange_norm = (series_elevrange_norm + a)**gamma + b*(series_elevrange_norm + a) + c
                    #  delta_h = (h_n + a)**gamma + b*(h_n + a) + c
                    # Limit the icethicknesschange_norm to between 0 - 1
                    #  (at the ends of the fxns may be slightly above 1 or below 0 because the fxns are not exact)
                    series_icethicknesschange_norm[series_icethicknesschange_norm > 1] = 1
                    series_icethicknesschange_norm[series_icethicknesschange_norm < 0] = 0
                    # remove off-glacier values
                    series_icethicknesschange_norm[series_elevrange_norm < 0] = 0
                    series_icethicknesschange_norm[series_elevrange_norm > 1] = 0
                    # Compute the annual glacier-wide volume change [km**3]
                    glacier_volumechange_annual = ((glac_bin_massbal_clim_mwe_annual.iloc[:, year_position] * 
                        glac_bin_area_annual.iloc[:, year_position]).sum() / 1000 * density_water / density_ice)
                    #  units: [m w.e.] * [km**2] * (1 km / 1000 m) * density_ice / density_water = km**3 ice
                    # Compute the Huss volume change scaling factor, fs_huss [km]                    
                    fs_huss = glacier_volumechange_annual / (glac_bin_area_annual.iloc[:, year_position] * 
                                                             series_icethicknesschange_norm).sum() * 1000
                    #  units: km**3 / (km**2 * [-]) * (1000 m / 1 km) = m ice
                    # Compute the ice thickness change [m ice] in each bin
                    series_icethicknesschange = series_icethicknesschange_norm * fs_huss
                    #  units: [-] * [km] * (1000 m / 1 km) = 0
                    # Compute the specific total mass balance [m w.e.]
                    #   i.e., the mass balance after the redistribution of ice 
                    glac_bin_massbal_total_mwe_annual.iloc[:, year_position] = (series_icethicknesschange * density_ice / 
                                                                                density_water)
                    #  units: [m ice] * [kg / (m_ice * m**2)] / [kg / (m_water * m**2)] = m w.e.
                    # if statement used to avoid error in final year, since model run is over
                    if glac_bin_icethickness_annual.columns.values[year_position] != endyear:
                        # Update the ice thickness [m] for the next year
                        glac_bin_icethickness_annual.iloc[:, year_position + 1] = (
                            glac_bin_icethickness_annual.iloc[:, year_position] + series_icethicknesschange)
                        # Update the volume [km**3] for the next year
                        glac_bin_volume_annual.iloc[:, year_position + 1] = (glac_bin_area_annual.iloc[:, year_position + 1] *
                            glac_bin_icethickness_annual.iloc[:, year_position + 1] / 1000)
                        #  units: [km**2] * [m] * (1 km / 1000 m) = km**3
            
            # ADD IN CAVEATS FROM HUSS AND HOCK (EX. MINIMUM OF 3 ELEVATION BINS)
            # ADD IN AREA CHANGES
            # NOTE: WANT TO HAVE OPTIONS TO USE THESE NORMALIZED ELEVATION CHANGE CURVES OR USE THE REAL THING!
                    
                                
            ###### Computing Glacier-wide Statistics ####################################
            # For calculating annual mass balance components below,
            #   year_position*annual_divisor gives initial step of the given year
            #   step gives the final step of the year and the "+1" is needed to include the final step (python is exclusive)
            # Annual glacier-wide specific accumulation [m w.e.]
            main_glac_acc_annual.iloc[glac, year_position] = (
                main_glac_acc_monthly.iloc[glac, year_position*annual_divisor:step+1].sum())
            # Annual glacier-wide specific refreeze [m w.e.]
            main_glac_refreeze_annual.iloc[glac, year_position] = (
                main_glac_refreeze_monthly.iloc[glac, year_position*annual_divisor:step+1].sum())
            # Annual glacier-wide specific melt [m w.e.]
            main_glac_melt_annual.iloc[glac, year_position] = (
                main_glac_melt_monthly.iloc[glac, year_position*annual_divisor:step+1].sum())
            # Annual glacier-wide specific frontal ablation [m w.e.]
            main_glac_frontal_ablation_annual.iloc[glac, year_position] = (
                main_glac_frontal_ablation_monthly.iloc[glac, year_position*annual_divisor:step+1].sum())
            # Annual glacier-wide specific climatic mass balance [m w.e.]
            main_glac_massbal_clim_mwe_annual.iloc[glac, year_position] = (
                main_glac_massbal_clim_mwe_monthly.iloc[glac, year_position*annual_divisor:step+1].sum())
            # Annual glacier-wide specific total mass balance [m w.e.]
            main_glac_massbal_total_mwe_annual.iloc[glac, year_position] = (
                (glac_bin_massbal_total_mwe_annual.iloc[:, year_position] * glac_bin_area_annual.iloc[:, year_position]).sum() /
                glac_bin_area_annual.iloc[:, year_position].sum())
            # Annual total glacier area [km**2]
            main_glac_area_annual.iloc[glac, year_position] = glac_bin_area_annual.iloc[:, year_position].sum()
            # Annual total glacier volume [km**3]
            main_glac_volume_annual.iloc[glac, year_position] = glac_bin_volume_annual.iloc[:, year_position].sum()
            # Annual Equilibrium Line Altitude (ELA) [m a.s.l.]
            series_massbal_clim_mwe_annual = glac_bin_massbal_clim_mwe_annual.iloc[:, year_position]
            # set up past value in case mass balance is zero over the entire glacier (acc = 0, temp < 0)
            if year_position == 0:
                ELA_annual_past = -9.99
            else:
                ELA_annual_past = main_glac_ELA_monthly.iloc[glac, year_position - 1]
            ELA_annual_value = massbalance.ELA_glacier(series_massbal_clim_mwe_annual, ELA_annual_past)
            main_glac_ELA_annual.iloc[glac, year_position] = ELA_annual_value
            # Annual Accumulation-Area Ratio (AAR) [%]
            #  Note: this function requires the ELA to be known
            series_area_annual = glac_bin_area_annual.iloc[:, year_position]
            main_glac_AAR_annual.iloc[glac, year_position] = massbalance.AAR_glacier(ELA_annual_value, series_area_annual, glac)            
            # Annual snowline [m a.s.l.]
            #   i.e., the transient snow line at time of minimum snow cover
            main_glac_snowline_annual.iloc[glac, year_position] = (
                main_glac_snowline_monthly.iloc[glac, year_position*annual_divisor:step+1].max())
            # Annual runoff [m**3]
            main_glac_runoff_annual.iloc[glac, year_position] = (
                main_glac_runoff_monthly.iloc[glac, year_position*annual_divisor:step+1].sum())

    # Record glacier bin specific output in the netcdf file
    # Monthly bin variables
    acc_bin_monthly[glac,:,:] = glac_bin_acc
    refreeze_bin_monthly[glac,:,:] = glac_bin_refreeze
    melt_bin_monthly[glac,:,:] = glac_bin_melt
    frontal_ablation_bin_monthly[glac,:,:] = glac_bin_frontal_ablation
    massbal_clim_mwe_bin_monthly[glac,:,:] = glac_bin_massbal_clim_mwe
    area_bin_monthly[glac,:,:] = glac_bin_area
    temp_bin_monthly[glac,:,:] = glac_bin_temp
    prec_bin_monthly[glac,:,:] = glac_bin_prec
    snowdepth_bin_monthly[glac,:,:] = glac_bin_snowdepth
    melt_snowcomponent_bin_monthly[glac,:,:] = glac_bin_melt_snowcomponent
    melt_glaccomponent_bin_monthly[glac,:,:] = glac_bin_melt_glaccomponent
    # Annual bin variables
    acc_bin_annual[glac,:,:] = glac_bin_acc_annual
    refreeze_bin_annual[glac,:,:] = glac_bin_refreeze_annual
    melt_bin_annual[glac,:,:] = glac_bin_melt_annual
    frontal_ablation_bin_annual[glac,:,:] = glac_bin_frontal_ablation_annual
    massbal_clim_mwe_bin_annual[glac,:,:] = glac_bin_massbal_clim_mwe_annual
    massbal_total_mwe_bin_annual[glac,:,:] = glac_bin_massbal_total_mwe_annual
    area_bin_annual[glac,:,:] = glac_bin_area_annual
    icethickness_bin_annual[glac,:,:] = glac_bin_icethickness_annual
    volume_bin_annual[glac,:,:] = glac_bin_volume_annual
    surftype_bin_annual[glac,:,:] = glac_bin_surftype_annual
    width_bin_annual[glac,:,:] = glac_bin_width_annual

# Record glacier-wide specific output in the netcdf file
# Monthly glacier variables
acc_glac_monthly[:,:] = main_glac_acc_monthly
refreeze_glac_monthly[:,:] = main_glac_refreeze_monthly
melt_glac_monthly[:,:] = main_glac_melt_monthly
frontal_ablation_glac_monthly[:,:] = main_glac_frontal_ablation_monthly
massbal_clim_mwe_glac_monthly[:,:] = main_glac_massbal_clim_mwe_monthly
area_glac_monthly[:,:] = main_glac_area_monthly
ELA_glac_monthly[:,:] = main_glac_ELA_monthly
AAR_glac_monthly[:,:] = main_glac_AAR_monthly
snowline_glac_monthly[:,:] = main_glac_snowline_monthly
runoff_glac_monthly[:,:] = main_glac_runoff_monthly
# Annual glacier variables
acc_glac_annual[:,:] = main_glac_acc_annual
refreeze_glac_annual[:,:] = main_glac_refreeze_annual
melt_glac_annual[:,:] = main_glac_melt_annual
frontal_ablation_glac_annual[:,:] = main_glac_frontal_ablation_annual
massbal_clim_mwe_glac_annual[:,:] = main_glac_massbal_clim_mwe_annual
massbal_total_mwe_glac_annual[:,:] = main_glac_massbal_total_mwe_annual
area_glac_annual[:,:] = main_glac_area_annual
volume_glac_annual[:,:] = main_glac_volume_annual
ELA_glac_annual[:,:] = main_glac_ELA_annual
AAR_glac_annual[:,:] = main_glac_AAR_annual
snowline_glac_annual[:,:] = main_glac_snowline_annual
runoff_glac_annual[:,:] = main_glac_runoff_annual

# # Close the netcdf file
# netcdf_output.close()

""" Determine how spinup years are handled """

print('Mass balance calculations have finished.')

elapsed = timeit.default_timer() - start_time
print(elapsed)

The 'downscaletemp2bins' function has finished.
The 'downscaleprec2bins' function has finished.
The 'accumulationbins' functions has finished.
The 'refreezingbins' functions has finished.


NameError: name 'timestep' is not defined

In [ ]:
# output_csvfilename = '../massbal2_total_2001_months.csv'
# output_df = glac_bin_massbal_total_mwe_annual.iloc[:,0:12].transpose()
# output_df.to_csv(output_csvfilename) 

In [47]:
# # Plot annual specific mass balance
# plt.figure(1)
# plt.plot(glac_bin_massbal_clim_mwe_annual.iloc[472:821,0], glac_bin_massbal_annual.index.values[472:821])
# plt.xlabel('Specific Mass Balance [m w.e.]')
# plt.ylabel('Elevation [m a.s.l.]')
# plt.title('Annual specific mass balance')

In [46]:
# Monthly outputs:
# print('Temperature:\n', glac_bin_temp.iloc[500:502, 0:12])
# print('Accumulation:\n', glac_bin_acc.iloc[500:502, 0:12])
# print('Refreeze:\n', glac_bin_refreeze.iloc[500:502, 0:12])
# print('Surface melt:\n', glac_bin_melt.iloc[500:502, 0:12])
# print('Specific mass balance:\n', glac_bin_massbal_clim_mwe.iloc[500:502,0:12])
# print('Snow Depth:\n', glac_bin_snowdepth.iloc[500:502, 0:12])

# # Monthly glacier-wide statistics
# print('Glacier-wide mass balance:\n', main_glac_massbal_clim_gt_monthly.iloc[:,0:12])
# print('Glacier-wide specific mass balance:\n', main_glac_massbal_clim_mwe_monthly.iloc[:,0:12])
# print('Monthly snow line altitude:\n', main_glac_SLA_monthly.iloc[:, 0:12])

# # Annual outputs:
# print('Annual temperature:\n', glac_bin_temp_annual.iloc[500:502, 0:5])
# print('Annual accumulation:\n', glac_bin_acc_annual.iloc[500:502, 0:5])
# print('Annual refreeze:\n', glac_bin_refreeze_annual.iloc[500:502, 0:5])
# print('Annual melt:\n', glac_bin_melt_annual.iloc[500:502, 0:5])
# print('Annual specific mass balance:\n', glac_bin_massbal_clim_mwe_annual.iloc[500:502, 0:5])
# print('Annual mass balance:\n', glac_bin_massbal_clim_gt_annual.iloc[500:502, 0:5])
# print('Annual surface type:\n', glac_bin_surftype_annual.iloc[555:570, 0:5])

# # Annual glacier-wide statistics
# print('Annual glacier-wide mass balance:\n', main_glac_massbal_clim_gt_annual)
# print('Annual glacier-wide specific mass balance:\n', main_glac_massbal_clim_mwe_annual)
# print('Annual snow line altitude:\n', main_glac_SLA_annual.iloc[:, 0:5])
# print('Annual glacier area:\n', main_glac_area_annual.iloc[:, 0:5])
# print('Annual glacier volume:\n', main_glac_volume_annual.iloc[:, 0:5])


Surface melt:
 date  2000-10-01  2000-11-01  2000-12-01  2001-01-01  2001-02-01  2001-03-01  \
5005         0.0           0           0           0           0           0   
5015         0.0           0           0           0           0           0   

date  2001-04-01  2001-05-01  2001-06-01  2001-07-01  2001-08-01  2001-09-01  
5005           0         0.0    0.854438    1.304314    1.113295    0.652743  
5015           0         0.0    0.837471    1.289806    1.098787    0.638703  


In [11]:
# output_filename = '../Output/PyGEM_output_rgiregion15_20171027.nc'

# # Open the netcdf
# output_dataset = nc.Dataset(output_filename)

# # # Print attributes and keys
# print('NetCDF format:', output_dataset.file_format)
# print('Description:', output_dataset.description)
# print('History:', output_dataset.history)
# print('Source:', output_dataset.source)
# print('Dimension keys:', output_dataset.dimensions.keys())
# # Print the details concerning each variable
# print('\nVariables:')
# for output_key in output_dataset.variables.keys():
#     output_print = output_dataset.variables[output_key]
#     print(output_key, output_print.dtype, output_print.dimensions, output_print.shape)

# # # Print out the content of a variable
# # print(output_dataset['snowline_firn_glac_annual'][:])
    
# output_dataset.close()

In [2]:
# Sample output functions
import pygemfxns_output as output
output_filename = '../Output/glac_bin_monthly_output_rgi15_20171016.nc'

# Seasonal runoff example
runoff_output = output.runoff_glacier(output_filename, 'seasonal', 0)
print('Seasonal runoff example:\n', runoff_output.iloc[:,0:5])
# Annual ELA example
ELA_output = output.ELA_glacier(output_filename, 'annual', 0)
print('Annual ELA example:\n', ELA_output.iloc[:,0:5])
# Annual AAR example
AAR_output = output.AAR_glacier(output_filename, 'annual', 0)
print('Annual AAR example:\n', AAR_output.iloc[:,0:5])

Seasonal runoff example:
 year            2001          2002          2003          2004          2005
summer  1.003775e+08  9.303592e+07  9.969952e+07  9.157220e+07  8.512340e+07
winter -3.210333e+06 -3.439835e+06 -3.702633e+06 -3.423238e+06 -3.682017e+06
Annual ELA example:
    2001  2002  2003  2004  2005
0  5670  5630  5630  5560  5580
Annual AAR example:
         2001       2002       2003       2004       2005
0  55.481387  56.075187  56.075187  57.471641  56.947459


In [24]:
# Extract user-specified output from the netcdf (TURN THIS INTO A FUNCTION)
# Input information:
output_filename = '../Output/glac_bin_monthly_output_rgi15_20171004.nc'
output_varname = 'massbal'
output_glacier = 0
output_dates_start = '2000-10'
output_dates_end = '2015-09'
output_binelev_start = 4000
output_binelev_end = 8500
output_option_plot = 0
#  > 1 = plot
#  > 0 = do not plot
output_option_writecsv = 0
#  > 1 = write data to csv file
#  > 0 = do not create file
output_csvfilename = ('../Output/' + 'glac_bin_' + output_varname + '_glacier' + str(output_glacier) + '_' 
                      + str(strftime("%Y%m%d")) + '.csv')

# Specify start of winter and start of summer months
winter_month_start = 10
#  winter is considered October 1 - April 30
summer_month_start = 5
#  summer is considered May 1 - Sept 30

# Convert elevation to bins
output_binelev_startindex = int(output_binelev_start / binsize)
output_binelev_endindex = int(output_binelev_end / binsize)

# Open the netcdf
output_dataset = nc.Dataset(output_filename)

# Print attributes and keys
print('NetCDF format:', output_dataset.file_format)
print('Description:', output_dataset.description)
print('History:', output_dataset.history)
print('Source:', output_dataset.source)
print('Dimension keys:', output_dataset.dimensions.keys())
# Print out the details concerning each variable
print('\nVariables:')
for output_key in output_dataset.variables.keys():
    output_print = output_dataset.variables[output_key]
    print(output_key, output_print.dtype, output_print.dimensions, output_print.shape)
    
# Extract data of interest
# Convert the dates back to a standard format of year-month
output_dates = pd.Series(nc.num2date(output_dataset['time'][:], units=output_dataset['time'].units, calendar="standard"))
# if timestep == 'monthly':
#     output_dates = output_dates.apply(lambda x: x.strftime('%Y-%m'))
# Find the start and end index position of the dates that will be used to extract the user-specified period of interest
output_dates_startindex = np.where(output_dates == output_dates_start)[0][0]
output_dates_endindex = np.where(output_dates == output_dates_end)[0][0]
# Select the bins, which will be used as an index
output_bins = pd.Series(output_dataset['binelev'][:])
# Create a dataframe with the data of interest
output_df = pd.DataFrame(output_dataset[output_varname][output_glacier, 
                                                        output_binelev_startindex:output_binelev_endindex+1, 
                                                        output_dates_startindex:output_dates_endindex+1], 
                         columns=output_dates[output_dates_startindex:output_dates_endindex+1], 
                         index=output_bins[output_binelev_startindex:output_binelev_endindex+1])

# Print sample of data
print(output_df.loc[5005:5025, '2000-10-01':'2001-09-01'])

# Compute glacier mass balance 

# Plot the output
if output_option_plot == 1:
    plt.figure(1)
    for plotbins in range(output_df.shape[0]):  
        plt.plot(output_df.columns.values, output_df.iloc[plotbins,:])
#     plt.legend()
    plt.xlabel('date')
    plt.ylabel(output_dataset[output_varname].standard_name + ' (' + output_dataset[output_varname].units + ')')
    plt.title(output_dataset[output_varname].standard_name + ' for glacier ' + str(output_glacier))
    
# Write the output to a csv file
if output_option_writecsv == 1:
    output_df.to_csv(output_csvfilename) 
    
# DEVELOP SCRIPT TO READ THROUGH THE NETCDF FILE AND PROCESS IT (ANNUAL, SUMMER, WINTER BALANCES, ETC.)
# HERE IS WHERE WE CAN ALSO CREATE WHATEVER PLOTS WE'D LIKE TO SEE ETC.
# ADDITIONAL COMPUTATIONS FOR ELA, AAR, RUNOFF, ETC.
output_dataset.close()

print('Output cell has finished.')

In [25]:
# # Plot annual glacier-wide mass balance
# plt.figure(1)
# plt.plot(main_glac_massbal_clim_mwe_annual.columns.values, main_glac_massbal_clim_mwe_annual.iloc[0,:])
# plt.plot(main_glac_massbal_clim_mwe_annual.columns.values, main_glac_massbal_clim_mwe_annual.iloc[1,:])
# plt.legend(('Ngozumpa', 'Khumbu'))
# plt.xlabel('Water Year')
# plt.ylabel('Mass Balance (m w.e.)')
# plt.title('Annual glacier-wide mass balance')

# # Plot annual specific mass balance versus elevation
# plt.figure(2)
# plt.plot(glac_bin_massbal_clim_mwe_annual.iloc[494:787, 0], glac_bin_massbal_clim_mwe_annual.index.values[494:787])
# plt.plot(glac_bin_massbal_clim_mwe_annual.iloc[494:787, 1], glac_bin_massbal_clim_mwe_annual.index.values[494:787])
# plt.plot(glac_bin_massbal_clim_mwe_annual.iloc[494:787, 13], glac_bin_massbal_clim_mwe_annual.index.values[494:787])
# plt.legend(('2001', '2002', '2014'))
# plt.xlabel('Mass Balance (m w.e.)')
# plt.ylabel('Elevation (m a.s.l.)')
# plt.title('Khumbu Glacier annual climatic mass balance versus elevation')

In [ ]:
# Aggregation to O1 and O2 RGI regions

In [ ]:
############################ SCRATCH CALCS BELOW HERE #################################################################

In [189]:
# for glac in [0]:
#     # Downscale the gcm temperature [degC] to each bin
#     glac_bin_temp = massbalance.downscaletemp2bins(main_glac_rgi, main_glac_hyps, gcm_glac_temp, gcm_glac_elev, glac)
# print(glac_bin_temp)

# batman = glac_bin_temp.values
# print(batman)
# print(glac_bin_temp.shape[1])
def annualweightedmean_array(var, dates_table):
    """
    Calculate annual mean of variable according to the timestep.
    Monthly timestep will group every 12 months, so starting month is important.
    """
    if input.timestep == 'monthly':
        dayspermonth = dates_table['daysinmonth'].values.reshape(-1,12)
        #  creates matrix (rows-years, columns-months) of the number of days per month
        daysperyear = dayspermonth.sum(axis=1)
        weights = (dayspermonth / daysperyear[:,np.newaxis]).reshape(-1)
        #  computes weights for each element, then reshapes it for next step
        var_annual = (var*weights[np.newaxis,:]).reshape(-1,12).sum(axis=1).reshape(-1,15)
        #  computes weighted average for each year
        print(daysperyear.shape[0])
    return var_annual

timestart_batman = timeit.default_timer()

# groupbyyearmean_array(glac_bin_temp.values, dates_table)
batman = annualweightedmean_array(glac_bin_temp.values, dates_table)

timeelapsed_batman = timeit.default_timer() - timestart_batman
print('Step time:', timeelapsed_batman)

15
Step time: 0.0017064332805603044
